In [82]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
from numpy import linalg as LA
import time
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import  KernelRidge
from sklearn.linear_model import  HuberRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic, Matern, ExpSineSquared,DotProduct
from sklearn.svm import SVR

from yellowbrick.features import ParallelCoordinates

In [83]:
#Reading Input

df1 = pd.read_csv('Input.csv')

df1.rename(columns = {'TLI':'InletLiquidTemperature', 'TAI':'InletAirTemperature'
                   , 'PI':'InletPressure', 'H':'PackedBedHeight'
                   , 'D':'PackedBedDiameter', 'ML':'LiquidMassFlow'
                   , 'MA':'AirMassFlow', 'EPS':'PackedBedPorosity'}, inplace = True)

X = df1.to_numpy() #Input

#Reading Output

df2 = pd.read_csv('Output.csv')

df2.rename(columns = {'TLO':'OutletLiquidTemperature', 'TAO':'OutletAirTemperature'
                   , 'PO':'OutletPressure'}, inplace = True)

Y = df2.to_numpy()
Y0 = Y[:,0]

Y.shape
n,m=X.shape


In [88]:
from sklearn.multioutput import MultiOutputRegressor
# Best linear estimation
time_start = time.time()
reg = MultiOutputRegressor(LinearRegression()).fit(X, Y)
time_elapsed = (time.time()-time_start)
print(time_elapsed)
Y_Lin_Predict=reg.predict(X)
MSE_Lin=mean_squared_error(Y,Y_Lin_Predict)
MAE_Lin=mean_absolute_error(Y,Y_Lin_Predict)
R2_Lin=r2_score(Y,Y_Lin_Predict)
Score_Lin=reg.score(X, Y)
print(MSE_Lin)
# print(MAE_Lin)
print(R2_Lin)
print(Score_Lin)

0.03989410400390625
4.352245206217219
0.9393768905157627
0.9393768905157627


In [65]:
kernel1 = RBF(10, (1e-3, 1e2))
kernel2 = RBF(5 , (1e-3, 1e2))
kernel3 = RationalQuadratic(length_scale=1.0, alpha=0.1)
kernel4 = Matern(length_scale=1.0, length_scale_bounds=(1e-05, 100000.0), nu=4.5)
kernel5 = ExpSineSquared(length_scale=2.0, periodicity=3.0,
                         length_scale_bounds=(1e-05, 100000.0),periodicity_bounds=(1e-05, 100000.0))
kernel6 = DotProduct()

gpr1 = GaussianProcessRegressor(kernel=kernel1, n_restarts_optimizer=1000,alpha=.1)
gpr2 = GaussianProcessRegressor(kernel=kernel2, n_restarts_optimizer=1000,alpha=.1)
gpr3 = GaussianProcessRegressor(kernel=kernel3, n_restarts_optimizer=1000,alpha=.1)
gpr4 = GaussianProcessRegressor(kernel=kernel4, n_restarts_optimizer=1000,alpha=.1)
gpr5 = GaussianProcessRegressor(kernel=kernel5, n_restarts_optimizer=1000,alpha=.1)
gpr6 = GaussianProcessRegressor(kernel=kernel6, n_restarts_optimizer=1000,alpha=.1)

In [66]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [67]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)
Y0_train = Y_train[:,0]
Y1_train = Y_train[:,1]
Y0_test = Y_test[:,0]
Y1_test = Y_test[:,1]

In [10]:
my_stand_scaler_obj = StandardScaler()
my_stand_scaler_obj.fit(X_train)

# rescale the training data
X_stand_scaler = my_stand_scaler_obj.transform(X)

# rescale the training data
X_train_stand_scaler = my_stand_scaler_obj.transform(X_train)

# also, rescale the training data
X_test_stand_scaler = my_stand_scaler_obj.transform(X_test)


print(my_stand_scaler_obj.mean_)

In [68]:
train_sizes, train_scores, test_scores = learning_curve(gpr1,
                                                        X_stand_scaler, Y0,
                                                        cv = 10,
                                                        scoring="neg_mean_squared_error", 
                                                        train_sizes = np.linspace(0.1, 0.9, 16))

# Means and standard deviations of training set scores
train_mean1 = np.mean(train_scores, axis=1)
# Means and standard deviations of test set scores
test_mean1 = np.mean(test_scores, axis=1)

NameError: name 'X_stand_scaler' is not defined

In [63]:
test_mean1

array([-6.77958391, -4.9778832 , -3.58576955, -2.46430201, -1.27962372,
       -1.10091842, -1.12537764, -1.06780471, -0.99226753, -0.35626691,
       -0.22682411, -0.22747015, -0.2144828 , -0.19163245, -0.17863533,
       -0.17119058])

In [24]:
train_sizes, train_scores, test_scores = learning_curve(gpr1,
                                                        X_stand_scaler, Y,
                                                        cv = 10,
                                                        scoring="r2",
                                                        train_sizes = np.linspace(0.1, 0.9, 16))

# Means and standard deviations of training set scores
train_mean2 = np.mean(train_scores, axis=1)
# Means and standard deviations of test set scores
test_mean2 = np.mean(test_scores, axis=1)

NameError: name 'X_stand_scaler' is not defined

In [72]:
gpr1 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel1, n_restarts_optimizer=1000,alpha=.1))
gpr1.fit(X_train_scaled, Y_train)
gpr1.score(X_train_scaled, Y_train)
gpr1.score(X_test_scaled, Y_test)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.

-488.9147320610609

In [ ]:
gpr1 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel1, n_restarts_optimizer=1000,alpha=.1))
gpr1.fit(X_train_scaled, Y_train)
gpr1.score(X_train_scaled, Y_train)
gpr1.score(X_test_scaled, Y_test)


In [ ]:
gpr1 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel4, n_restarts_optimizer=1000,alpha=.1))
gpr1.fit(X_train_scaled, Y_train)
gpr2=gpr1.score(X_train_scaled, Y_train)
gpr3=gpr1.score(X_test_scaled, Y_test)

In [ ]:
gpr1 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel5, n_restarts_optimizer=1000,alpha=.1))
gpr1.fit(X_train_scaled, Y_train)
gpr2=gpr1.score(X_train_scaled, Y_train)
print(gpr2)
gpr3=gpr1.score(X_test_scaled, Y_test)
print(gpr3)

In [ ]:
gpr1 = MultiOutputRegressor(GaussianProcessRegressor(kernel=kernel6, n_restarts_optimizer=1000,alpha=.1))
gpr1.fit(X_train_scaled, Y_train)
gpr2=gpr1.score(X_train_scaled, Y_train)
print(gpr2)
gpr3=gpr1.score(X_test_scaled, Y_test)
print(gpr3)

In [74]:


Y_GPR_Predict=gpr1.predict(X_test_scaled)
MSE_GPR=mean_squared_error(Y_test,Y_GPR_Predict)
MAE_GPR=mean_absolute_error(Y_test,Y_GPR_Predict)
R2_GPR=mean_absolute_error(Y_test,Y_GPR_Predict)
Score_GPR=gpr1.score(X_test_scaled,Y_test)

print(MSE_GPR)

ValueError: y_true and y_pred have different number of output (3!=1)